In [1]:
using Combinatorics
using DoubleFloats
using IterTools
using LinearAlgebra
using Random
using RandomMatrices
using SpecialFunctions
using Test

In [2]:
"""
    random_covariance(N; hbar=2, pure=false)

Generate random covariance matrix of a Gaussian state.

### Input
- `N`  -- Int: Number of modes
- `hbar` -- Float: (optional, default: `hbar=2`) value of ``\\hbar`` in the commutation relation ``[x,p]=i\\hbar``
- `pure` -- Bool: (optional, default: `pure=false`) if true, generates a matrix corresponding to a pure Gaussian state
### Output

Array: ``2N \\times 2N`` random covariance matrix.
"""
function random_covariance(N; hbar=2, pure=false)
    U = rand(Haar(2), N)
    O = [real(U) -imag(U); imag(U) real(U)]
    U = rand(Haar(2), N) 
    P = [real(U) -imag(U); imag(U) real(U)]
    r = abs.(randn(N))
    sq = diagm(vcat(exp.(-r), exp.(r)))
    S = O * sq * P 
    if pure 
        return (hbar / 2) .* (S * transpose(S))
    else
        nbar = 2 .* (rand(N)) .+ 1
        V = (hbar / 2) * diagm(vcat(nbar, nbar))
        return S * V * transpose(S)  
    end
end;

In [3]:
"""
    q_mat(cov; hbar=2)

Compute the Husimi covariance matrix of a Gussian state.

### Input
- `cov`  -- Array: ``2N \\times 2N`` ``xp``-Wigner covariance matrix of the Gaussian state
- `hbar` -- Float: (optional, default: `hbar=2`) value of ``\\hbar`` in the commutation relation ``[x,p]=i\\hbar``

### Output

Array: complex Husimi covariance matrix of the Gaussian state.
"""
function q_mat(cov; hbar=2)
    n = size(cov)[1] ÷ 2
    x = (2 / hbar) .* cov[1:n, 1:n]
    xp = (2 / hbar) .* cov[1:n, n + 1:end]
    p = (2 / hbar) .* cov[n + 1:end, n + 1:end]
    ad = (x + p + im .* (xp - transpose(xp)) - 2 .* I) ./ 4
    a = (x - p + im .* (xp + transpose(xp))) ./ 4
    return [ad conj(a); a conj(ad)] + I
end;

In [4]:
"""
    gen_omats(l, nbar; dtype=ComplexDF64)

Generates the matrix O that enters inside the Torontonian for an l mode system
in which the first mode is prepared in a thermal state with mean photon number nbar
and the rest in vacuum and are later sent into a Fourier interferometer, i.e. one described
by a DFT unitary matrix.

### Input
- `l`    -- Int: number of modes
- `nbar` -- Float: mean photon number of the first mode (the only one not prepared in vacuum)
- `dtype`-- (optional, default: `dtype=ComplexDF64`) type of the output matrix 

### Output

Array: an O matrix whose Torontonian can be calculated analytically.
"""
function gen_omats(l, nbar; dtype=ComplexDF64)
    A = (nbar / (l * (1.0 + nbar))) * ones(dtype, l, l)
    O = [A 0*A ; 0*A A]
    return O
end;

In [5]:
"""
    analytical_tor(l, nbar)

Return the value of the Torontonian of the O matrices generated by gen_omats.

### Input
- `l`    -- Int: number of modes
- `nbar` -- Float: mean photon number of the first mode (the only one not prepared in vacuum)

### Output

Float: value of the torontonian of gen_omats(l, nbar).
"""
function analytical_tor(l, nbar)
    if isapprox(l, nbar, atol=1e-14, rtol=0)
        return 1.0
    end
    beta = -(nbar / (l * (1 + nbar)))
    pref = factorial(l) / beta
    p1 = (pref * l * gamma(1 / beta)) / gamma(1 / beta + l + 2)
    p2 = (pref * beta * gamma(2 + 1 / beta)) / gamma(2 + 1 / beta + l)
    return (p1 + p2) * ((-1) ^ l)
end;

In [6]:
"""
    quad_cholesky(L, Z, idx, mat)

Returns the Cholesky factorization of a matrix using sub-matrix of prior
Cholesky based on the new matrix and lower right quadrant.

Algorithm from paper:
https://arxiv.org/pdf/2109.04528.pdf

### Input
- `L`   -- Array: previous Cholesky factorization
- `Z`   -- Array: new sub-matrix indices
- `idx` -- Int: index of starting row/column of lower right quadrant
- `mat` -- Array: new matrix

### Output

Array: the Cholesky factorization of matrix `mat`.
"""
function quad_cholesky(L, Z, idx, mat)
    L = Array{ComplexDF64}(L)
    mat = Array{ComplexDF64}(mat)
    Ls = L[Z, Z]
    lmat = size(mat)[1]
    for i in idx:1:lmat
        for j in idx:1:i-1
            z = 0.0
            for k in 1:1:j-1
                z += Ls[i, k] * conj(Ls[j, k])
            end
            Ls[i, j] = (mat[i, j] - z) / Ls[j, j]
        end
        z = 0.0
        for k in 1:1:i-1
            z += Ls[i, k] * conj(Ls[i, k])
        end
        Ls[i, i] = real(sqrt(mat[i, i] - z))
    end
    return Ls
end;

In [7]:
"""
    recursiveTor(L, modes, A, n)

Returns the recursive Torontonian sub-computation of a matrix.

Algorithm from paper:
https://arxiv.org/pdf/2109.04528.pdf

### Input
- `L`     -- Array: current Cholesky factorization
- `modes` -- Array: optical modes
- `A`     -- Array: a square, symmetric array of even dimensions
- `n`     -- Int: size of the original matrix 

### Output

Complex: the recursive Torontonian sub-computation of matrix `A`.
"""
function recursiveTor(L, modes, A, n)
    L = Array{ComplexDF64}(L)
    A = Array{ComplexDF64}(A)
    tot = 0.0
    if length(modes) == 0
        start = 1
    else
        start = modes[end] + 1
    end
    for i in start:1:n
        nextModes = vcat(modes, i)
        nm = size(A)[1] ÷ 2
        idx = 2 * (i - length(modes)) - 1
        Z = vcat(collect(1:1:(idx - 1)), collect((idx + 2):1:(2 * nm)))
        Az = A[Z, Z]
        Ls = quad_cholesky(L, Z, idx, I - Az)
        det = prod(diag(Ls)) .^ 2 
        tot += ((-1) ^ length(nextModes)) / sqrt(det) + recursiveTor(Ls, nextModes, Az, n)
    end
    return tot
end;

In [8]:
"""
    tor(O)

Returns the Torontonian of a matrix (using directly the definition).

### Input
- `O` -- Array: a square, symmetric array of even dimensions.

### Output

Complex: the Torontonian of matrix `O`.
"""
function tor(O)
    O = Array{ComplexDF64}(O)
    n = size(O)[1] ÷ 2
    total = 0.0
    for set in powerset(1:n)
        pm_coeff = (-1) ^ (length(set))
        kept_rows = vcat(set, set .+ n)
        btt = sqrt(det(I - O[kept_rows, kept_rows]))
        total += pm_coeff / btt
    end
    return ((-1) ^ n) * total
end;

In [9]:
"""
    rec_tor(A)

Returns the Torontonian of a matrix (using the recursive algorithm).

Algorithm from paper:
https://arxiv.org/pdf/2109.04528.pdf

### Input
- `A` -- Array: a square, symmetric array of even dimensions.

### Output

Complex: the Torontonian of matrix `A`.
"""
function rec_tor(A)
    A = Array{ComplexDF64}(A)
    n = size(A)[1] ÷ 2
    Z = Array{Int64}(undef, 2 * n)
    Z[1:2:end] = collect(1:1:n)
    Z[2:2:end] = collect((n + 1):1:(2 * n))
    A = A[Z, Z]
    L = cholesky(I - A).L
    det = prod(diag(L)) .^ 2
    return 1 / sqrt(det) + recursiveTor(L, Array{Int64}(undef, 0), A, n) 
end;

In [10]:
"""
    threshold_detection_prob(cov, det_pattern; hbar=2, recursive=true)

Returns threshold detection probabilities for Gaussian states without displacement.

### Input
- `cov`         -- Array: 2D xp Wigner covariance matrix
- `det_pattern` -- Array: 1D array of {0,1} that describes the threshold detection outcome
- `hbar`        -- Float: (optional, default: `hbar=2`) value of ``\\hbar`` in the 
                          commutation relation ``[x,p]=i\\hbar``
- `recursive`   -- Bool: (optional, default: `recursive=true`) indicates whether to use the 
                        recursive algorithm for the computation of the Torontonian or not

### Output

Float: probability of the detction pattern.
"""
function threshold_detection_prob(cov, det_pattern; hbar=2, recursive=true)
    cov = Array{ComplexDF64}(cov)
    n = size(cov)[1] ÷ 2
    qcov = q_mat(cov, hbar=hbar)
    O = I - inv(qcov)
    rpatt = vcat(det_pattern, det_pattern)
    rows = findall(x -> x != 0, rpatt)
    rO = O[rows, rows]
    if recursive
        return rec_tor(Hermitian(rO)) / sqrt(det(qcov))
    else
        return tor(rO) / sqrt(det(qcov))
    end
end;

In [11]:
#checking that the Torontonian function (from definition) gives the correct analytical results
@testset "Torontonian (from definition) vs. analytical result" begin
    for i = 2:1:13
        for k = 1.1:0.37:7.3
            @test isapprox(analytical_tor(i, k), tor(gen_omats(i, k)))
        end
    end
end;

Test Summary:                                       | Pass  Total
Torontonian (from definition) vs. analytical result |  204    204


In [12]:
#checking that the Torontonian function (recursive) gives the correct analytical results
@testset "Torontonian (recursive) vs. analytical result" begin
    for i = 2:1:13
        for k = 1.1:0.37:7.3
            @test isapprox(analytical_tor(i, k), rec_tor(gen_omats(i, k)))
        end
    end
end;

Test Summary:                                 | Pass  Total
Torontonian (recursive) vs. analytical result |  204    204


In [13]:
#Computing threshold detection probabilities
N = 4 #number of modes
cov = random_covariance(N) #rendom covariance matrix
println("pattern", " | ", "prob (definition)", " | ", "prob (recursive)")
for p in product([[0 1] for i = 1:N]...) #this line generates all possible detection patterns
    pattern = collect(p)
    prob_df = threshold_detection_prob(cov, pattern, recursive = false)
    prob_rc = threshold_detection_prob(cov, pattern, recursive = true)
    println(pattern, " | ", real(prob_df), " | ", real(prob_rc))
end

pattern | prob (definition) | prob (recursive)
[0, 0, 0, 0] | 0.06507496410834357 | 0.06507496410834357
[1, 0, 0, 0] | 0.0034324592075896045 | 0.0034324592075896045
[0, 1, 0, 0] | 0.009999227670283547 | 0.009999227670283547
[1, 1, 0, 0] | 0.038959279561609404 | 0.038959279561609404
[0, 0, 1, 0] | 0.013435752178618783 | 0.013435752178618783
[1, 0, 1, 0] | 0.026273905343936928 | 0.026273905343936914
[0, 1, 1, 0] | 0.009760054407607618 | 0.009760054407607618
[1, 1, 1, 0] | 0.2426663801923161 | 0.2426663801923156
[0, 0, 0, 1] | 0.021998987775370214 | 0.021998987775370214
[1, 0, 0, 1] | 0.002473645468522954 | 0.0024736454685229544
[0, 1, 0, 1] | 0.01266520930503906 | 0.012665209305039062
[1, 1, 0, 1] | 0.02337750291328705 | 0.02337750291328706
[0, 0, 1, 1] | 0.01602701184944208 | 0.016027011849442083
[1, 0, 1, 1] | 0.02009929406904555 | 0.020099294069045523
[0, 1, 1, 1] | 0.020021633341405824 | 0.020021633341405824
[1, 1, 1, 1] | 0.4737346926075817 | 0.4737346926075797
